In [1]:
import numpy as np
import matplotlib.pyplot as plt
import leidenalg as la
import igraph as ig

In [34]:
class temporal_network:
    def __init__(self, size, length, data, **kwargs):
        
        if length < 2: return('Object should be a multilayer network with at least 2 layers')
        if size < 3: return('Layers must have at least 3 nodes')
        
        self.size = size # number of nodes in every layer
        self.length = length # number of layers
        self.nodes = [i for i in range(self.size)]
        
        #### data: you can go back and forth between supra adjacency and list adjacency but not to edge list
        
        ##         if supra__adjacency: creates the list adjacency matrix
        ##
        ##                     --- additional arguments ---
        ##                       - supra_adjacency: supra adjacency matrix of shape (size*time x size*time)
        
        
        ##
        ##         if edge__list: creates undirected binary multilayer network from the egde triplets
        ##                      given of the form (i,j,t). supra_adjacency and list_adjacency matrices 
        ##                      are automatically created. 
        ##
        ##                     --- additional arguments ---
        ##                       - edge_list: list of triplets e.g. [(0,2,1),(2,1,1),(0,1,2),(0,2,2)]
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind: if temporal, only adjacent layers gets connected with strength 'omega'
        ##                               if multilayer, all layers get connected w/ each other w/ strength 'omega'
        
        
        ##         if list__adjacency: creates the supra adjacency matrix
        ##              
        ##                     --- additional arguments ---
        ##                       - list_adjacency: list of length 'length' that contains individual adjacency
        ##                                         matrices of each layer
        ##                       - omega: interlayer coupling strength, can be a float only for now
        ##                       - kind : if temporal, only adjacent layers gets connected w/ strength 'omega'
        ##                                if multilayer, all layers get connected w/ each other w/ strength 'omega'
        
                    
        if  data == 'supra__adjacency':
            self.supra_adjacency = kwargs['supra_adjacency']
            list_adjacency = np.zeros((self.length, self.size, self.size))
            
            for i in range(self.length-1):
                list_adjacency[i][i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size] = self.supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size]
            
            self.list_adjacency = list_adjacency
        
        elif data == 'edge__list':
            self.edgelist = kwargs['edge_list']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            list_adjacency = np.zeros((self.length, self.size, self.size))
            
            for k,e in enumerate(self.edge_list):
                i,j,t = e[0],e[1],e[2]
                supra_adjacency[self.size*(t-1)+i][self.size*(t-1)+j] = 1
                supra_adjacency[self.size*(t-1)+j][self.size*(t-1)+i] = 1
                list_adjacency[t-1][i][j] = 1
                list_adjacency[t-1][j][i] = 1
        
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'temporal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'multilayer':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency
            self.list_adjacency = list_adjacency
            
        elif data == 'list__adjacency':
            self.list_adjacency = kwargs['list_adjacency']
            supra_adjacency = np.zeros((self.size*self.length,self.size*self.length))
            
            for i in range(self.length):
                supra_adjacency[i*self.size:(i+1)*self.size,i*self.size:(i+1)*self.size] = self.list_adjacency[i]
            
            ##filling off-diagonal blocks
            if kwargs['kind'] == 'temporal':
                for n in range(self.size*(self.length-1)):
                    supra_adjacency[n][n+self.size] = kwargs['omega']
                    supra_adjacency[n+self.size][n] = kwargs['omega']
                
            elif kwargs['kind'] == 'multilayer':
                i = 0
                while self.length-i != 0:
                    i = i+1
                    for n in range(self.size*(self.length-i)):
                        supra_adjacency[n][n+i*self.size] = kwargs['omega']
                        supra_adjacency[n+i*self.size][n] = kwargs['omega']
            
            self.supra_adjacency = supra_adjacency

In [92]:
class temporal_network:
    
    #node-aligned
    #get binary edge pairs
    def __init__(self, data, length, edge_list, kind = 'temporal', omega = 1):
        
        self.nodes = [i for i in range(size)]
        self.time = length
        self.size = size
        self.edgelist = edge_list
        adjacency = np.zeros((self.size*self.time,self.size*self.time))
        multi_array = np.zeros((self.time,self.size,self.size))
        
        #edge list is binary in the format of (i,j,t): 
        #there is an edge between i and j on time t(and an edge between j and i)
        for k,e in enumerate(edge_list):
            i,j,t = e[0],e[1],e[2]
            adjacency[self.size*(t-1)+i][self.size*(t-1)+j] = 1
            adjacency[self.size*(t-1)+j][self.size*(t-1)+i] = 1
            multi_array[t-1][i][j] = 1
            multi_array[t-1][j][i] = 1
            
        ##filling off diagonal blocks
        if kind == 'temporal':
            for n in range(self.size*(self.time-1)):
                adjacency[n][n+self.size] = omega
                adjacency[n+self.size][n] = omega
                
        elif kind == 'multilayer':
            i = 0
            while self.time-i != 0:
                i = i+1
                for n in range(self.size*(self.time-i)):
                    adjacency[n][n+i*self.size] = omega
                    adjacency[n+i*self.size][n] = omega
                    
        self.supra_adjacency = adjacency
        self.multi_array = multi_array
        
    def aggragate(self, normalized = True):
        t = self.time 
        n = self.size
        aggragated = np.zeros((n,n))
        for i,c in enumerate(self.multi_array[:]):
            aggragated = aggragated + c
        if normalized: return (aggragated/t)
        else: return (aggragated)
        
    def modularity_matrix(self, omega, gamma):
        N = self.size
        T = self.time
        B = np.zeros((N*T,N*T))
        two_mu = 0
        for i in range(T):
            k = np.sum(self.multi_array[i],0)
            two_m = np.sum(k,0)
            two_mu = two_mu + two_m
            B[i*N:(i+1)*N,i*N:(i+1)*N] = self.multi_array[i] - (gamma * k.T*k)/(two_m)
        two_mu = two_mu + 2*omega*N*(T-1)
        for p in range(N*(T-1)):
            B[p][p+N] = omega 
            B[p+N][p] = omega
        return(B)
    
    def edgelist2edges(self):
        T = self.time
        all_edges = [[] for i in range(T)]
        for k,e in enumerate(np.sort(self.edgelist, axis=-1)):
            i,j,t = e[0], e[1], e[2]
            pair = (i,j)
            all_edges[t-1].append(pair)
        return (all_edges)
    
    def create_igraph(self, omega):
        T = self.time
        N = self.size
        G = []
        for i in range(T):
            G.append(ig.Graph())
            G[i].add_vertices(N)
            G[i].add_edges(self.edgelist2edges()[i])
            G[i].vs['id'] = list(range(N))
            G[i].vs['node_size'] = 0
            
        G_coupling = ig.Graph.Formula('1 -- 2 -- 3 -- 4');
        G_coupling.es['weight'] = omega; # Interslice coupling strength
        G_coupling.vs['slice'] = [G[0], G[1], G[2], G[3]]
        
        layers, interslice_layer, G_full = la.slices_to_layers(G_coupling, 
                                                               slice_attr = 'slice', 
                                                               vertex_id_attr='id', 
                                                               weight_attr='weight');
        
        partitions = [la.CPMVertexPartition(H, 
                                            node_sizes='node_size', 
                                            weights='weight', 
                                            resolution_parameter=omega) for H in layers];
        
        interslice_partition = la.CPMVertexPartition(interslice_layer, 
                                                     resolution_parameter=omega, 
                                                     node_sizes='node_size', 
                                                     weights='weight');
        return(partitions, interslice_partition)
    
    def create_igraph_try(self, omega):
        T = self.time
        N = self.size
        G = []
        for i in range(T):
            G.append(ig.Graph())
            G[i].add_vertices(N)
            G[i].add_edges(self.edgelist2edges()[i])
            G[i].vs['id'] = list(range(N))
            G[i].vs['node_size'] = 0
            
        G_coupling = ig.Graph.Formula('1 -- 2 -- 3 -- 4');
        G_coupling.es['weight'] = omega; # Interslice coupling strength
        G_coupling.vs['slice'] = [G[0], G[1], G[2], G[3]]
        
        layers, interslice_layer, G_full = la.time_slices_to_layers([G[0],G[1],G[2],G[3]],interslice_weight=0.1);
        partitions = [la.CPMVertexPartition(H, node_sizes='node_size',
                                            weights='weight', resolution_parameter=omega) for H in layers];
        interslice_partition = la.CPMVertexPartition(interslice_layer, resolution_parameter=omega)
                                                     
        return(partitions, interslice_partition)
    
    def leiden_try(self):
        optimiser = la.Optimiser()
        diff = optimiser.optimise_partition_multiplex(self.create_igraph_try(omega=0)[0] + [self.create_igraph_try(omega=0)[1]])

        return(diff)
    
    def leiden(self):
        optimiser = la.Optimiser()
        diff = optimiser.optimise_partition_multiplex(self.create_igraph(omega=0)[0] + [self.create_igraph(omega=0)[1]])
        return(diff)

In [94]:
edges = [[0,1,1],[0,1,2],[1,2,2],[0,1,3],[0,2,3],[0,1,4],[0,2,4],[1,2,4]]
T = temporal_network(3, 4, edges, kind = 'temporal', omega = 1)
S = temporal_network(3, 4, edges, kind = 'temporal', omega = 1).supra_adjacency
C = temporal_network(3, 4, edges, kind = 'temporal', omega = 1).multi_array
E = temporal_network(3, 4, edges, kind = 'temporal', omega = 1).edgelist2edges()
#G = temporal_network(3, 4, edges, kind = 'temporal', omega = 1).create_igraph()
#print(T.create_igraph()[0])

In [97]:
diff_try = T.leiden_try()
part, int_part = T.create_igraph_try(omega = 0)
print(int_part,part)

Clustering with 12 elements and 12 clusters
[ 0] 0
[ 1] 1
[ 2] 2
[ 3] 3
[ 4] 4
[ 5] 5
[ 6] 6
[ 7] 7
[ 8] 8
[ 9] 9
[10] 10
[11] 11 [<leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9950>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9a10>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9c90>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9d50>]


In [98]:
diff = T.leiden()
part, int_part = T.create_igraph(omega = 0)
print(int_part,part)

Clustering with 12 elements and 12 clusters
[ 0] 0
[ 1] 1
[ 2] 2
[ 3] 3
[ 4] 4
[ 5] 5
[ 6] 6
[ 7] 7
[ 8] 8
[ 9] 9
[10] 10
[11] 11 [<leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c93d0>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c91d0>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9e90>, <leidenalg.VertexPartition.CPMVertexPartition object at 0x7fb4ed6c9810>]
